In [32]:
from qiskit.circuit.library import ZZFeatureMap



def FeatureMapBuilder(num_features, reps):
    feature_map = ZZFeatureMap(
        feature_dimension=num_features,
        reps=reps,
    )
    return feature_map

my_feature_map = FeatureMapBuilder(3, 2)
# my_feature_map.decompose().draw("mpl", style="clifford")

In [33]:
from qiskit.circuit.library import TwoLocal

def AnsatzBuilder(num_qubits, reps):
    ansatz = TwoLocal(
        num_qubits=num_qubits,
        rotation_blocks='ry',
        entanglement_blocks='cx',
        entanglement='full', 
        reps=reps, 
        insert_barriers=True
    )
    return ansatz

my_ansatz = AnsatzBuilder(5, 3)
# my_ansatz.decompose().draw("mpl")